In [22]:
import pandas as pd
import numpy as np

URL = {'customer':'https://assets.datacamp.com/production/repositories/1646/datasets/c3a701a4729471ae0b92d8c300b470fd2ec0a73a/user_demographics_v1.csv',
       'purchase':'https://assets.datacamp.com/production/repositories/1646/datasets/5decd183ef3710475958bbc903160fd6354379d5/purchase_data_v1.csv',
       'revenue':'https://assets.datacamp.com/production/repositories/1646/datasets/3afb49cad9fb91c02b71b52a2ddc0071ea13764c/daily_revenue.csv'
      }
customer=pd.read_csv(URL['customer'])
purchase=pd.read_csv(URL['purchase'])
revenue=pd.read_csv(URL['revenue'])

### clean customer data
# convert uid to integer type
customer.uid = customer.uid.astype('int')
# truncate reg_date column to date
customer.reg_date = pd.to_datetime(customer.reg_date).dt.date

### clean purchase data
# convert date column to datetime
purchase.date = pd.to_datetime(purchase.date)

### clean revenue data
#convert date to datetime and truncate to date
revenue.date = pd.to_datetime(revenue.date).dt.date

display(customer.head())
display(purchase.head())
display(revenue.head())

,uid,reg_date,device,gender,country,age
0,54030035,2017-06-29,and,M,USA,19
1,72574201,2018-03-05,iOS,F,TUR,22
2,64187558,2016-02-07,iOS,M,USA,16
3,92513925,2017-05-25,and,M,BRA,41
4,99231338,2017-03-26,iOS,M,FRA,59


,date,uid,sku,price
0,2017-07-10,41195147,sku_three_499,499
1,2017-07-15,41195147,sku_three_499,499
2,2017-11-12,41195147,sku_four_599,599
3,2017-09-26,91591874,sku_two_299,299
4,2017-12-01,91591874,sku_four_599,599


,index,gender,device,revenue,date,country
0,1,male,ios,186557.0,2013-09-20,USA
1,2,male,ios,177217.0,2013-09-21,USA
2,3,male,ios,177197.0,2013-09-22,USA
3,4,male,ios,157193.0,2013-09-23,USA
4,5,male,ios,165592.0,2013-09-24,USA


In [26]:
# Combine purchase and customer data
df=purchase.merge(customer, on=['uid'], how='inner')
# Summary statistics for price and age
purchase_summary = df.agg({'price':['mean','median'],
                            'age':['mean','median']})
print(purchase_summary)
# Summary statistics for price by device and gender
grouped_data = df.groupby(by=['device','gender'])
purchase_summary = grouped_data.agg({'price':['mean','median','std']})
print(purchase_summary)

             price        age
mean    406.772596  23.922274
median  299.000000  21.000000
                    price                   
                     mean median         std
device gender                               
and    F       400.747504    299  179.984378
       M       416.237308    499  195.001520
iOS    F       404.435330    299  181.524952
       M       405.272401    299  196.843197
